<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# The IBM Q Provider 

In Qiskit we have an interface for backends and jobs that is useful for running circuits and extending to third-party backends. In this tutorial, we will review the core components of Qiskit’s base backend framework, using the IBM Q Provider as an example.

The interface has four main component: the account, providers, backends, and jobs:

- account: Your account with IBM Q.
- providers: Access to services such as backends.
- backends: Real quantum devices or simulators.
- jobs: A local reference for keeping track of computations on backends.

## The Provider

The IBM Q Provider is an entity that provides access to groups of different backends (for example, backends available through the open IBM Q Experience, or specific groups in the IBM Q Network).

The IBM Q Provider main point of entry is as `qiskit.IBMQ`. This is an object that allows obtaining `provider` instances, corresponding to your IBM Q Experience capabilities:

- `providers()`: returns all the provider objects in your account.
- `get_provider(hub, group, project)`: returns the specified provider.

Additionally, it has some extra functions for handling administrative tasks. The credentials can be saved to disk or used in a session and never saved. 

- `enable_account(token, url)`: enable the account in the current session
- `disable_account(**kwargs)`: disable the accounts from current session
- `save_account(token)`: save the account to disk
- `delete_account(**kwargs)`: delete the account from disk
- `load_account(**kwargs)`: load previously-saved account into session
- `active_account()`: list the account active in this session
- `stored_account()`: list the account saved to disk

The `provider` objects obtained through `get_provider()` inherit from BaseProvider and implement the methods:

- `backends()`: returns all backend objects known to the provider.
- `get_backend(name)`: returns the named backend.


<div class="alert alert-block alert-info">
<b>Note:</b> The use of `provider` instances is the default way of retrieving backends from Qiskit 0.11 onwards - if you have been using earlier versions of Qiskit, check the "Updating from previous versions" section for more detailed instructions on updating and using the different options.</div>
</div>

In [1]:
from qiskit import IBMQ 

IBMQ.providers()

[]

Here we see that there are no providers. This is because no accounts have been loaded.

Let's start fresh and delete any accounts on disk. If no accounts are on disk, this will error

In [2]:
IBMQ.delete_account()

verify that there are no accounts stored now

In [3]:
IBMQ.stored_account()

{}

To enable an account (useful for one-off use, or if you don't want to save to disk)

In [4]:
# Change this to be your API Token
my_api_token = 'CHANGE_THIS_TO_YOUR_API_TOKEN'
provider = IBMQ.enable_account(my_api_token)

RequestsApiError: 401 Client Error: Unauthorized for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. LOGIN_FAILED, Error code: LOGIN_FAILED.

The `IBMQ.enable_account()` and `IBMQ.load_account()` methods will automatically return the provider for the open IBM Q Experience, for convenience. To see what providers you have available

In [ ]:
IBMQ.providers()

and backends which are available in the default provider

In [ ]:
provider.backends()

Disable that account (so we go back to no accounts active)

In [ ]:
IBMQ.disable_account()

Now no providers are available

In [ ]:
IBMQ.providers()

For convenience, you can save your account to disk:

In [ ]:
IBMQ.save_account(my_api_token, overwrite=True)

Now it should show up as present on disk

In [ ]:
# uncomment to print to screen (it will show your token and url)
# IBMQ.stored_account()

but no account active in current session yet

In [ ]:
IBMQ.active_account()

now load up the account stored to disk

In [ ]:
provider = IBMQ.load_account()

If you have access to multiple hubs, you can obtain a provider for each of them using `.get_provider()`:

In [ ]:
provider_1 = IBMQ.get_provider(hub='open')
provider_1.backends()

In [ ]:
provider_2 = IBMQ.get_provider(hub='ibm-q-internal')
provider_2.backends()

## Filtering the backends

You may also optionally filter the set of returned backends, by passing arguments that query the backend's `configuration`, `status`, or `properties`. The filters are passed by conditions, and for more general filters you can make advanced functions using the lambda function.

As a first example: only return currently operational devices

In [ ]:
provider.backends(operational=True, simulator=False)

only return backends that are real devices, have at most 5 qubits, and are operational

In [ ]:
provider.backends(filters=lambda x: x.configuration().n_qubits <= 5 and 
              not x.configuration().simulator and x.status().operational==True)

Filter: show the least busy device (in terms of pending jobs in the queue)

In [ ]:
from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits == 5 and
                                                       not x.configuration().simulator)
least_busy(small_devices)

The above filters can be combined as desired.

If you just want to get an instance of a particular backend, you can use the `get_backend()` method.

In [ ]:
provider.get_backend('ibmq_16_melbourne')

## The backend

Backends represent either a simulator or a real quantum computer, and are responsible for running quantum circuits and returning results. Their `run` method takes in a `qobj` as input, which is a quantum object and the result of the compilation process; it returns a BaseJob object. This object allows asynchronous running of jobs for retrieving results from a backend when the job is completed.

At a minimum, backends use the following methods, inherited from BaseBackend:


- `provider` - returns the provider of the backend.
- `name()` - gets the name of the backend.
- `status()` - gets the status of the backend.
- `configuration()` - gets the configuration of the backend.
- `properties()` - gets the properties of the backend.
- `run()` - runs a qobj on the backend.

For remote backends, they must also support

- `jobs()` - returns a list of previous jobs executed by this user on this backend.
- `retrieve_job()` - returns a job by a job_id.

In future updates they will introduce the following commands

- `defaults()` - gives a data structure of typical default parameters.
- `schema()` - gets a schema for the backend

There are some IBM Q-only functions 

- `hub`  - returns the IBM Q hub for this backend.
- `group` - returns the IBM Q group for this backend.
- `project` - returns the IBM Q project for this backend.

In [ ]:
backend = least_busy(small_devices)

Let's start with the `backend.provider`, which returns a provider object

In [ ]:
backend.provider

Next is the `name()`, which returns the name of the backend

In [ ]:
backend.name()

Next let's look at the `status()`:

    operational lets you know that the backend is taking jobs
    pending_jobs lets you know how many jobs are in the queue

In [ ]:
backend.status()

The next is `configuration()`

In [ ]:
backend.configuration()

The next is `properties()` method

In [ ]:
backend.properties()

The next is `hub`, `group`, and `project`. For the IBM Q experience these will return `None`

In [ ]:
backend.hub

In [ ]:
backend.group

In [ ]:
backend.project

To see your last 5 jobs run on the backend, use the `jobs()` method of that backend

In [ ]:
for ran_job in backend.jobs(limit=5):
    print(str(ran_job.job_id()) + " " + str(ran_job.status()))

Then the job can be retreived using the `retrieve_job(job_id())` method

In [ ]:
job = backend.retrieve_job(ran_job.job_id())

## The Job object

Job instances can be thought of as the “ticket” for a submitted job. They find out the execution’s state at a given point in time (for example, if the job is queued, running, or has failed), and allow control over the job. They have the following methods:

- `status()` - returns the status of the job.
- `backend()` - returns the backend the job was run on.
- `job_id()` - gets the job_id.
- `cancel()` - cancels the job.
- `result()` - gets the results from the circuit run.

IBM Q-only functions 

- `creation_date()` - gives the date at which the job was created.
- `queue_position()` - gives the position of the job in the queue.
- `error_message()` - gives the error message of failed jobs.

Let's start with the `status()`. This returns the job status and a message

In [ ]:
job.status()

To get a backend object from the job, use the `backend()` method

In [ ]:
backend_temp = job.backend()
backend_temp

To get the job_id use the `job_id()` method

In [ ]:
job.job_id()

To get the result from the job, use the `result()` method

In [ ]:
result = job.result()
counts = result.get_counts()
print(counts)

If you want to check the creation date, use `creation_date()`

In [ ]:
job.creation_date()

Let's make an active example

In [ ]:
from qiskit import *
from qiskit.compiler import transpile, assemble

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

To compile this circuit for the backend, use the compile function. It will make a qobj (quantum object) that can be run on the backend using the `run(qobj)` method. 

In [ ]:
qobj = assemble(transpile(circuit, backend=backend), shots=1024)
job = backend.run(qobj)

The status of this job can be checked with the `status()` method

In [ ]:
job.status()

If you made a mistake and need to cancel the job, use the `cancel()` method.

In [ ]:
import time
#time.sleep(10)

job.cancel()

The `status()` will show that the job cancelled. 

In [ ]:
job.status()

To rerun the job and set up a loop to check the status and queue position, use the `queue_position()` method. 

In [ ]:
job = backend.run(qobj)

In [ ]:
from qiskit.tools.monitor import job_monitor
job_monitor(job)
result = job.result()

In [ ]:
counts = result.get_counts()
print(counts)

## Updating from previous versions

Since July 2019 (and with `Qiskit` version `0.11`), the IBM Q Provider defaults to using the new [IBM Q Experience](https://quantum-computing.ibm.com), which supersedes the legacy Quantum Experience and Qconsole.

If you have credentials for the legacy Quantum Experience or Qconsole stored in disk, you can make use of `IBMQ.update_account()` helper. This helper will read your current credentials stored in disk and attempt to convert them:


In [ ]:
# IBMQ.update_account()

You can find more information on how to update your programs in the [README.md](https://github.com/Qiskit/qiskit-ibmq-provider/blob/master/README.md#updating-to-the-new-ibm-q-experience) file in the provider repository.